In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import xml.etree.ElementTree as ET
import pickle
import math
import copy
from rdp import *
token = "pk.eyJ1IjoibG1hZ25hbmEiLCJhIjoiY2s2N3hmNzgwMGNnODNqcGJ1N2l2ZXZpdiJ9.-aOxDLM8KbEQnJfXegtl7A"
px.set_mapbox_access_token(token)
n_voxel = 3

In [2]:
def load_gpx():
    with open('gpx.df','rb') as infile:
        df = pickle.load(infile)
    begin = int(df.iloc[-1]["route_num"])
    print(begin)
    for i in range(begin+1, begin+1109+1):
        tree = ET.parse('Datas/GPS/GPX/data'+str(i)+'.gpx')
        if(len(tree.getroot()) > 1):
            root = tree.getroot()[1][0]
            df_temp = pd.DataFrame(columns=['lat', 'lon'])
            j=0
            for child in root:
                coord = child.attrib
                coord['lat'] = float(coord['lat'])
                coord['lon'] = float(coord['lon'])
                df_temp = df_temp.append(pd.DataFrame(coord, index=[j]))
                j+=1
            df_temp["route_num"] = i
            df = df.append(df_temp)
    with open('gpx.df', 'wb') as outfile:
        pickle.dump(df, outfile)

In [3]:
def rd_compression(df):
    df_simplified = pd.DataFrame(columns=['lat', 'lon', 'route_num'])
    for i in range(1, 1110):
        route = df[df['route_num']==i].values
        if(len(route)>0):
            simplified = rdp(np.delete(route, 2, 1), epsilon=1e-4)
            simplified = np.insert(simplified, 2, route[0][2], axis=1)
            df_temp = pd.DataFrame(simplified, columns=['lat', 'lon', 'route_num'])
            df_simplified = df_simplified.append(df_temp)
    return df_simplified

In [4]:
def display(dfdisplay, n=75, line_group="route_num", color=None):
    n+=1
    if(type(dfdisplay) == str):
        with open(dfdisplay,'rb') as infile:
            df = pickle.load(infile)
            dfdisplay = df[df["route_num"]<n]
    fig = px.line_mapbox(dfdisplay, lat="lat", lon="lon", line_group=line_group, color=color, zoom=11)
    fig.show()

In [11]:
#display("gpx_simplified.df", 1)

In [6]:
import math
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

In [7]:
def find_voxel_int(p, n=n_voxel):
    return [math.trunc(p[0]*10**n), math.trunc(p[1]*10**n)]

In [8]:
def find_line(p1, p2):
    if((p2[0]-p1[0]) != 0):
        line_coeff = (p2[1]-p1[1])/(p2[0]-p1[0])
    else:
        line_coeff = p1[0]
    b_line = p1[1]-line_coeff*p1[0]
    return [line_coeff, b_line]

def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return [x, y]
    

In [92]:

def create_dict_vox(df, nb_routes):
    dict_vox = {}
    #d = n+1
    for route in range(1, nb_routes+1):
        
        points = df_simplified[df_simplified["route_num"]==route].values.tolist()
        l=[]

        for j in range(len(points)-1):
            p1 = points[j]
            p2 = points[j+1]

            if(p1[0]>p2[0]):
                lat_orientation = -1
            else:
                lat_orientation = 1

            if(p1[1]>p2[1]):
                lon_orientation = -1
            else:
                lon_orientation = 1

            line = find_line(p1, p2)

            vox_int = find_voxel_int(p1)
            vox_final_int = find_voxel_int(p2)

            while(vox_int[0] != vox_final_int[0] or vox_int[1] != vox_final_int[1]):
                vox_float = [vox_int[0]*10**-n_voxel, vox_int[1]*10**-n_voxel]
                
                key = str(vox_int[0])+";"+str(vox_int[1])
                if key in dict_vox:
                    if(route not in dict_vox[key]):
                        dict_vox[key].append(route)
                else :
                    dict_vox[key] = [route]

                if(lat_orientation>0):
                    intersection_lat = line_intersection([p1, p2], [[vox_float[0]+1*10**-n_voxel, vox_float[1]],
                                                        [vox_float[0]+1*10**-n_voxel, vox_float[1]+1*10**-n_voxel]])
                else:
                    intersection_lat = line_intersection([p1, p2], [vox_float, [vox_float[0], vox_float[1]+1*10**-n_voxel]])


                if(lon_orientation>0):
                     intersection_lon = line_intersection([p1, p2], [[vox_float[0], vox_float[1]+1*10**-n_voxel], 
                                                        [vox_float[0]+1*10**-n_voxel, vox_float[1]+1*10**-n_voxel]])
                else:
                    intersection_lon = line_intersection([p1, p2], [vox_float, [vox_float[0]+1*10**-n_voxel, vox_float[1]]])

                intersection_lon_distance = sqrt((p1[0]-intersection_lon[0])**2+(p1[1]-intersection_lon[1])**2)
                intersection_lat_distance = sqrt((p1[0]-intersection_lat[0])**2+(p1[1]-intersection_lat[1])**2)

                if(intersection_lat_distance<intersection_lon_distance):
                    vox_int[0] += lat_orientation

                else:
                    vox_int[1] += lon_orientation
                    
            key = str(vox_int[0])+";"+str(vox_int[1])
            if key in dict_vox:
                if(route not in dict_vox[key]):
                    dict_vox[key].append(route)
            else :
                 dict_vox[key] = [route]
    return dict_vox


    


In [98]:
with open("gpx_simplified.df",'rb') as infile:
        df_simplified = pickle.load(infile)
        
        
        
nb_routes = 50
routes_min_quadrillage =

d = create_dict_vox(df_simplified, 50)

tab_vox = []
num_vox = 0
for key in d:
    if(len(d[key]) >= 10):
        vox_str = key.split(";")
        vox_float = [int(vox_str[0])*10**-n_voxel, int(vox_str[1])*10**-n_voxel]
        vox_float.append(num_vox)
        vox_float.append(1)
        tab_vox.append(vox_float)
        tab_vox.append([vox_float[0]+1*10**-n_voxel, vox_float[1], num_vox, 1])
        tab_vox.append([vox_float[0]+1*10**-n_voxel, vox_float[1]+1*10**-n_voxel, num_vox, 1])
        tab_vox.append([vox_float[0], vox_float[1]+1*10**-n_voxel, num_vox, 1])
        num_vox -= 1
        

df_simplified = df_simplified[df_simplified["route_num"]<=50]
df_simplified["type"] = 0
df = pd.DataFrame(tab_vox, columns=["lat", "lon", "route_num", "type"])
df_simplified = df_simplified.append(df)
display(df_simplified, color="type")        
        